In [3]:
sc = SparkContext.getOrCreate()

In [4]:
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
data = sc.textFile("trainIdx2_matrix.txt")
test_album = sc.textFile("test_album.txt")
test_artist = sc.textFile("test_artist.txt")
test_track = sc.textFile("test_track.txt")
ratings = data.map(lambda l: l.split('|')).map(lambda l: Rating(int(l[0]), int(l[1]), float(l[2])))
testData_track=test_track.map(lambda l: l.split(',')).map(lambda l: (int(l[0]), int(l[1])))
testData_album=test_album.map(lambda l: l.split(',')).map(lambda l: (int(l[0]), int(l[1])))
testData_artist=test_artist.map(lambda l: l.split(',')).map(lambda l: (int(l[0]), int(l[1])))

In [6]:
rank = 10
numIterations = 10
model = ALS.train(ratings, rank, numIterations)

In [11]:
predictions_track = model.predictAll(testData_track).map(lambda r: ((r[0], r[1]), r[2])).sortByKey()
predictions_album = model.predictAll(testData_album).map(lambda r: ((r[0], r[1]), r[2])).sortByKey()
predictions_artist = model.predictAll(testData_artist).map(lambda r: ((r[0], r[1]), r[2])).sortByKey()

In [8]:
def toCSVLine(data):
  return ','.join(str(d) for d in data)

In [9]:
lines = predictions_track.map(toCSVLine)
lines.repartition(1).saveAsTextFile("MF_Result_track")

In [ ]:
lines = predictions_album.map(toCSVLine)
lines.repartition(1).saveAsTextFile("MF_Result_album")

In [ ]:
line = predictions_artist.map(toCSVLine)
line.repartition(1).saveAsTextFile("MF_Result_artist")

In [6]:
# Evaluate the model on training data
trainFortest = ratings.map(lambda p: (p[0], p[1]))
predictions = model.predictAll(trainFortest).map(lambda r: ((r[0], r[1]), r[2]))
ratesAndPreds = ratings.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)
MSE = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
print("Mean Squared Error = " + str(MSE))

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 210.0 failed 1 times, most recent failure: Lost task 2.0 in stage 210.0 (TID 149, localhost, executor driver): java.io.FileNotFoundException: C:\Users\wenbi\AppData\Local\Temp\blockmgr-fc837d6e-f9e8-4426-90c3-4f9b390a3649\18\temp_shuffle_206d24d2-48fd-46a3-bb28-c5bfc6a77d47 (The system cannot find the path specified)
	at java.io.FileOutputStream.open0(Native Method)
	at java.io.FileOutputStream.open(Unknown Source)
	at java.io.FileOutputStream.<init>(Unknown Source)
	at org.apache.spark.storage.DiskBlockObjectWriter.initialize(DiskBlockObjectWriter.scala:102)
	at org.apache.spark.storage.DiskBlockObjectWriter.open(DiskBlockObjectWriter.scala:115)
	at org.apache.spark.storage.DiskBlockObjectWriter.write(DiskBlockObjectWriter.scala:229)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWri